# 

# Automating processes:
----
(interactive version -> *.ipynb*)

## Importing libraries:

In [1]:
import numpy as np
import pandas as pd
import selenium as sn
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import os

load_dotenv()

link = os.getenv('LINK')
user_login = os.getenv('USER_LOGIN')
user_pw = os.getenv('USER_PASSWORD')

Setting up driver for Google Chrome:

In [2]:
def fill_value_with_id(driver, id, value):
    try:
        driver.execute_script("""
            var campo = arguments[0];
            var valor = arguments[1];
            campo.value = valor;

            // Trigger the change event
            var changeEvent = new Event('change', { bubbles: true });
            campo.dispatchEvent(changeEvent);
        """, driver.find_element(By.ID, id), value)

        # Wait for the ASP.NET element to reload
        WebDriverWait(driver, 10).until(
            EC.staleness_of(driver.find_element(By.ID, id))
        )
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, id))
        )

        # Wait until page is ready and fully loaded
        WebDriverWait(driver, 10).until(
        lambda driver: driver.execute_script('return document.readyState') == 'complete'
        )
    except Exception as e:
        print("Ocorreu um erro ao preencher o campo:\n", e)

In [3]:
# Checks if the webdriver is installed and up to date
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Opens the link stored in .env
driver.get(link)

# Fill the login field
login = driver.find_element(By.ID, 'txtLogin')
login.send_keys(user_login)

# Fill the password field
login = driver.find_element(By.ID, 'txtSenha')
login.send_keys(user_pw)

# Click the enter button
enter_btn = driver.find_element(By.ID, 'btnOk')
enter_btn.click()

# Enter the 'Recuperação Ambiental' module and 'Controle' submodule
module_btn = driver.find_element(By.ID, 'imbModuloP3')
driver.execute_script("arguments[0].click();", module_btn)
sub_module_btn = driver.find_element(By.ID, 'imbModuloP3')
sub_module_btn = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "div.rollfx[onclick*='DefaultRecuperacaoAmbientalControle.aspx']"))
)
driver.execute_script("arguments[0].click();", sub_module_btn)

# Wait until the new page is completely loaded
WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script('return document.readyState') == 'complete'
)

# Enter the module 'Controle > Boletim de Silvicultura'
# Click on first element ('Controle') and wait until the 
# submodule ('Boletim de Silvicultura') is visible, then click it
first_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "td.MenuItem[id='Menu1-menuItem000']"))
)
first_element.click()

# Wait until the second element is visible and clickable
second_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "td.MenuItem[id='Menu1-menuItem000-subMenu-menuItem000']"))
)
second_element.click()

# Wait until the new page is completely loaded
WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script('return document.readyState') == 'complete'
)

# time.sleep(10)

# Wait for the iframe to load
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'conteudo'))
)

# Switch context to iframe
driver.switch_to.frame(iframe)

# Wait until the element is loaded and visible
radio_button = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, 'rblApontamento_0'))
)

# Using Javascript to trigger the 'click' method
driver.execute_script("""
    var radio = arguments[0];
    radio.checked = true;
    var event = new Event('click', { bubbles: true });
    radio.dispatchEvent(event);
""", radio_button)

# Wait until the new page is completely loaded
WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script('return document.readyState') == 'complete'
)
time.sleep(3)
# Wait until the href element is loaded and ready
table = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, 'olvOrdemServico'))
)
tbody = table.find_element(By.TAG_NAME, "tbody")
even_rows = tbody.find_elements(By.CSS_SELECTOR, "tr.even")
cell = even_rows[0].find_elements(By.TAG_NAME, "td")[1]
link = cell.find_element(By.TAG_NAME, "a")
link.click()

# Wait until the new page is completely loaded
WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script('return document.readyState') == 'complete'
)

time.sleep(3)

WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script('return document.readyState') == 'complete'
)

fill_value_with_id(driver, 'txtRegiao', '11')
# time.sleep(3)
fill_value_with_id(driver, 'txtProjeto', '026')
# time.sleep(3)
fill_value_with_id(driver, 'txtTalhao', 'V01651')
# time.sleep(3)
fill_value_with_id(driver, 'txtOperacao', '1036')
# time.sleep(3)
fill_value_with_id(driver, 'txtProjInv', '3')